# [LVV-T2190 (v1.0)] MTAOS add aberrations to M1M3+M2+Hexapods

This notebook is used to execute the [LVV-T2190 (v1.0)] test script during System Spread Integration Tests on Level 3.  
Execution steps are separated by horizontal lines.   
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.  

**Requirements:**
 - All the MT components should be enabled.
 
**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.
The log messages printed in this notebook are stored in the EFD Script using the `TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[LVV-T2190 (v1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2190
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

***

## Prepare the environment

Load all the needed libraries.
Get the remotes ready
Code in the notebook including section: "Check the summary state of each CSC".

In [ ]:
test_case = "LVV-T2190"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from astropy import time
from lsst.ts import utils

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

Use the `maintel/setup_mtcs.py` script in LOVE or the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(logging.ERROR)

In [ ]:
await mtcs.start_task

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [ ]:
vandv.logger.add_filter_to_mtcs()

The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [ ]:
script = salobj.Controller("Script", index=vandv.get_index(test_case))
await asyncio.sleep(10) 

script.start_task

***

## Ready M1M3

- Raise mirror 
- Turn on FB 
- Clear forces
- Need to have M1M3 LUT use its inclinometer.

In principle, this should be done above when running the setup notebook or the setup script.

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED, 
    components=["mtm1m3"],
    overrides=dict(mtm1m3="Default")
)

In [ ]:
await mtcs.raise_m1m3()

In [ ]:
await mtcs.enable_m1m3_balance_system()

In [ ]:
await mtcs.reset_m1m3_forces()

In [ ]:
script.log.info(f"{test_case} - {test_exec} - M1M3 ready")

***

## Ready M2
- Turn on FB
- Clear forces
- Need to have M2 LUT use its inclinometer

In principle, this should be done above when running the setup notebook or the setup script.

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED, 
    components=["mtm2"],
)

In [ ]:
await mtcs.enable_m2_balance_system()

In [ ]:
await mtcs.reset_m2_forces()

In [ ]:
script.log.info(f"{test_case} - {test_exec} - M2 ready")

***

## Get CamHex ready
- Check config; 
- Make sure LUT is on, and has valid inputs; 
- Make sure hex is at LUT position

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED, 
    components=["mthexapod_1"],
)

In [ ]:
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

In [ ]:
await mtcs.reset_camera_hexapod_position()

In [ ]:
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_1)

In [ ]:
script.log.info(f"{test_case} - {test_exec} - CamHex ready")

***

## Get CamHex ready
- Check config; 
- Make sure LUT is on, and has valid inputs; 
- Make sure hex is at LUT position

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED, 
    components=["mthexapod_1"],
)

In [ ]:
await vandv.hexapod.get_hexapod_configuration(mtcs.rem.mthexapod_1)

In [ ]:
await vandv.hexapod.check_hexapod_lut(mtcs.rem.mthexapod_1)

In [ ]:
await mtcs.reset_camera_hexapod_position()

In [ ]:
await vandv.hexapod.print_hexapod_compensation_values(mtcs.rem.mthexapod_1)
await vandv.hexapod.print_hexapod_position(mtcs.rem.mthexapod_1)

In [ ]:
script.log.info(f"{test_case} - {test_exec} - CamHex ready")

---
## Slew to a target and track.

Choose a target such that the rotator stays within a couple of degrees of its initial position. (az = 120º, el = 60º, rotator type = PhysicalSky, rot=1.9º)

This is because the CCW might not be running (MTmount in simulation mode).

In [ ]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

In [ ]:
script.log.info(f"{test_case} - {test_exec} - Slewed to target: {target}")

***

## Add 1um of z7 to the system via OFC

Compare the corrections sent vs forces and position changes applied. 
This is currently done in a separate notebook.

In [ ]:
script.log.info(f"{test_case} - {test_exec} - START")

# The wavefront errors input is an array with 19 elements where the first 
#   element corresponds to z4
wavefront_errors = np.zeros(19)

# Add 1 um to z7
wavefront_errors[3] += 1.0 # add1 um to z7

script.log.info(f"{test_case} - {test_exec} - Add 1um aberration")
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

script.log.info(f"{test_case} - {test_exec} - Issue 1um aberration")
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
## Analyse changes in the aberrations

Make plots using telemetry from each component to verify the changes in the DoFs.

***
## Reset the corrections using the resetCorrection command

Compare the corrections sent vs forces and position changes applied (these are all expected to be zero).  
This is currently done in a separate notebook or on Chronograf. 

In [ ]:
script.log.info(f"{test_case} - {test_exec} - Reset corrections")
await mtcs.rem.mtaos.cmd_resetCorrection.start(timeout=60.)

script.log.info(f"{test_case} - {test_exec} - Issue reset corrections")
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
## Add 2um of z7 to the system via OFC

Compare the corrections sent vs forces and position changes applied.  
This is currently done in a separate notebook or on Chronograf. 

In [ ]:
# Set 2 um to z7
wavefront_errors[3] = 2.0

script.log.info(f"{test_case} - {test_exec} - Add 2um aberration")
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

script.log.info(f"{test_case} - {test_exec} - Issue 2um aberration")
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
## Stop Tracking

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.reset_m1m3_forces()
await mtcs.reset_m2_forces()
await mtcs.reset_camera_hexapod_position()
await mtcs.reset_m2_hexapod_position()
script.log.info(f"{test_case} - {test_exec} - STOP")

***
Check that the corrections in step 10 are twice of those in step 7. This step does not currently involve running any commands in this notebook. This step must be verified using a separate noteboook. 

In [ ]:
await mtcs.point_azel(az=0, el=80)
await mtcs.stop_tracking()

***
Wrap up. Put each component to the following states:
mtaos --> standby
m1m3 --> lower mirror --> standby
m2 --> standby
camera hex --> standby
m2 hex --> standby

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()